In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import cv2
import numpy
from matplotlib import pyplot as plt
import glob
import json
import os
from mpl_toolkits.axes_grid1 import ImageGrid
import csv
import json
import os
from PIL import Image
from io import BytesIO
import base64
import pandas as pd
import torch
import torch.nn.functional as F
import numpy as np
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
from fairseq.dataclass.utils import convert_namespace_to_omegaconf
from tasks.mm_tasks.refcoco import RefcocoTask
from models.ofa import OFAModel
from PIL import Image, ImageOps
import re

In [ ]:
test_set = 'vqa_rad_open.tsv'
fp = open(os.path.join('vqa_data/', test_set), "r")
fp.seek(0)
images = []
gts = []
questions = []
images2 = []
fp.seek(0)
while(True):
    column_l = fp.readline().rstrip("\n").split("\t")
    if len(column_l) == 1: break
    images.append(column_l[1])
    gts.append(column_l[2])
    questions.append(column_l[3])
    images2.append(column_l[4])

In [ ]:
exp_name = 'coqah_5_vqa_rad_open.json'
res_dict = json.loads(open(exp_name, 'r').read())
res = res_dict["res"]
gts = res_dict["gts"]

replaced_gts = [re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", x.lower().replace("x-ray", "xray").replace("x ray", "xray").replace("radiography", "xray").replace("radiograph", "xray").replace("cxr", "chest xray").replace("pa view", "pa").replace("right side", "right").replace("left side", "left")) for x in gts]

In [ ]:
# Basic ChatGPT class
import openai

class BasicChatGPT:
    def __init__(self, openai_api_key, max_tokens=100):
        openai.api_key = openai_api_key
        self.max_tokens = max_tokens
        self.messages = []

    def call(self):
        response = openai.ChatCompletion.create(
          model="gpt-4",
          messages=self.messages,
          max_tokens=self.max_tokens,
          seed = 123,
        )
        return response

    def prompt(self, text):
        # Update the messages so far
        self.messages.append({
            "role": "user",
            "content": text,
        })

        # Call ChatGPT
        response = self.call()

        # Save the returned message
        message = response["choices"][0]["message"]
        self.messages.append(message)

        return message["content"]

In [ ]:
STARTING_PROMPT = """ 
You are given a question, a set of gold-standard reference answers written by experts, and a candidate answer. Please rate the accuracy of the candidate answer for the question considering the reference answers. Use a scale of 1-3, with 1 indicating an incorrect or irrelevant answer, 2 indicating an ambiguous or incomplete answer, and 3 indicating a correct answer. Give the rationale before rating. THIS IS VERY IMPORTANT: A binary question should only be answered with 'yes' or 'no', otherwise the candidate answer is incorrect.

Question: 'What is the color of the car?'
Reference answers: 'red', 'red', 'red', 'red', 'scarlet'
Candidate answer: 'pink'
Output: The candidate answer is incorrect because the car is 'red' and not 'pink'. So rating = 1

Question: 'What is the animal on the left?'
Reference answers: 'elephant', 'giraffe', 'giraffe', 'giraffe', 'giraffe'
Candidate answer: 'giraffe'
Output: The candidate answer is correct because most of the reference answers (4 out of 5) indicate the animal on the left is a giraffe. So rating = 3

Question: 'What’s the weather like?'
Reference answers: 'bright', 'bright and sunny', 'clear', 'sunny', 'sunny', 'sunny'
Candidate answer: 'cloudy'
Output: The candidate answer is incorrect because the weather is 'bright' and 'sunny', not cloudy. So rating = 1

Question: 'What are the people in the picture doing?'
Reference answers: 'sitting', 'sitting', 'sitting', 'sitting'
Candidate answer: 'they are resting'
Output: The candidate answer is ambiguous because, while it is common that people who are sitting are resting, it is not always the case. So rating = 2

Question: 'What color are the base tiles?'
Reference answers: 'beige', 'beige', 'beige', 'brown', 'brown', 'tan', 'tan', 'tan', 'tan', 'ten'
Candidate answer: 'brown'
Output: The candidate answer is correct because the reference answers include 'brown' and other similar colors such as 'tan' or 'beige'. So rating = 3

Question: 'How many people are in the picture?'
Reference answers: 'four', 'three', 'three', 'three', 'two', 'two'
Candidate answer: 'a few'
Output: The candidate answer is incomplete because 'a few' is less specific than the numerical reference answers. So rating = 2

Question: 'What type of fruit is in the picture?'
Reference answers: 'apple'
Candidate answer: 'fruit'
Output: The candidate answer is incorrect because it does not specify the type of fruit. So rating = 1

Question: 'What type of sculpture is this?'
Reference answers: 'Horse statue.'
Candidate answer: 'horse'
Output: The candidate answer is correct because 'horse' is equivalent to 'horse statue' in this context. So rating = 3
"""

In [ ]:
save_file = exp_name.replace('', 'lave_')

if os.path.isfile(save_file):
    lave_res = json.loads(open(save_file, 'r').read())
    starting_idx = len(lave_res)
else:
    lave_res = []
    starting_idx = 0

for idx in range(starting_idx, len(gts)):
    lave_res_dict = {}
    test_prompt = f"\n\nQuestion: '{questions[idx]}'\nReference answers: '{replaced_gts[idx]}'\nCandidate answer: '{res[idx]}'\nOutput: "
    prompt = STARTING_PROMPT + test_prompt
    openai_api_key = "<YOUR_OPENAI_API_KEY>"
    chatgpt = BasicChatGPT(openai_api_key)
    response = chatgpt.prompt(prompt)
    rate_idx = response.find('rating = ') + len('rating = ')
    rating = response[rate_idx:rate_idx+1]
    score = (int(rating)-1)/2
    lave_res_dict["id"] = idx
    lave_res_dict["question"] = questions[idx]
    lave_res_dict["gt"] = replaced_gts[idx]
    lave_res_dict["res"] = res[idx]
    lave_res_dict["response"] = response
    lave_res_dict["rating"] = rating
    lave_res_dict["score"] = score
    lave_res.append(lave_res_dict)
    with open(save_file, 'w') as f:
        json.dump(lave_res, f)
    print(idx)